In [95]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import os
# import validators
import requests
from tqdm import tqdm

## evaluation-tables.json

- sind die Entitäten (task, dataset, paper) die offenbar nur als
Strings gegeben sind, eindeutig auf deren URLs (z.B.
https://paperswithcode.com/task/image-classification) zurückzuführen?
- wie sieht die category-Struktur für die Tasks aus?
- gibt es eine Überschneidung zwischen model_name-Angaben und method?
- ist die Sub-task-hierarchie eine Baumstruktur oder gibt es Loops o.Ä.?

Den Code dazu am besten von Anfang an auch im Repo
https://gitlab.hzdr.de/tarek.saier/sc_combi_hiwi
dann kann ich im Zweifelsfall auch schon Zwischenergebnisse usw. sehn.

In [2]:
with open('/home/ls3data/datasets/paperswithcode/evaluation-tables.json') as f:
    evaluation_tables = json.load(f)

In [3]:
len(evaluation_tables)

1280

In [48]:
evaluation_tables[0].keys()

dict_keys(['categories', 'subtasks', 'source_link', 'task', 'description', 'datasets', 'synonyms'])

In [5]:
# 1. Categories -- what does it mean?
# according to the website, the only category for optical character reg is ComVis
categories = evaluation_tables[0]["categories"]
print(categories)

['Methodology', 'Natural Language Processing', 'Computer Vision']


In [6]:
# 2. subtasks
# "substasks" is a list that contains 8 elements
# each element is a dict contains the same keys as each single element in evaluation_tables
substasks = evaluation_tables[0]["subtasks"]
len(substasks)

8

In [7]:
evaluation_tables[0]["subtasks"][0].keys()

dict_keys(['categories', 'subtasks', 'source_link', 'task', 'description', 'datasets', 'synonyms'])

In [32]:
# name of the first subtask
evaluation_tables[0]["subtasks"][0]['task']

'Active Learning'

In [36]:
# the subtask of the first subtask
evaluation_tables[0]["subtasks"][1]["subtasks"]

[]

In [8]:
# 3. source link
# no info about source link (same for subtasks)
source_link = evaluation_tables[0]["source_link"]
source_link == None

True

In [9]:
# 4. task
task = evaluation_tables[0]["task"]
task

'Optical Character Recognition'

In [10]:
# 5. description
description = evaluation_tables[0]["description"]
description

'Optical character recognition or optical character reader (OCR) is the electronic or mechanical conversion of images of typed, handwritten or printed text into machine-encoded text, whether from a scanned document, a photo of a document, a scene-photo (for example the text on signs and billboards in a landscape photo, license plates in cars...) or from subtitle text superimposed on an image (for example: from a television broadcast)'

In [11]:
# 6. datasets
datasets = evaluation_tables[0]["datasets"]
print(len(datasets))
print("\n")
print(datasets[0])
print("\n")
print(datasets[0].keys())

3


{'dataset_links': [{'url': 'https://paperswithcode.com/sota/optical-character-recognition-on-i2l-140k', 'title': 'Papers with Code Leaderboard URL'}], 'description': '', 'sota': {'metrics': ['BLEU'], 'rows': [{'metrics': {'BLEU': '89.09%'}, 'paper_date': '2018-02-15', 'model_links': [], 'uses_additional_data': False, 'code_links': [{'url': 'https://github.com/untrix/im2latex', 'title': 'untrix/im2latex'}], 'paper_title': 'Teaching Machines to Code: Neural Markup Generation with Visual Attention', 'model_name': 'I2L-NOPOOL', 'paper_url': 'http://arxiv.org/abs/1802.05415v2'}, {'metrics': {'BLEU': '89%'}, 'paper_date': '2018-02-15', 'model_links': [], 'uses_additional_data': False, 'code_links': [{'url': 'https://github.com/untrix/im2latex', 'title': 'untrix/im2latex'}], 'paper_title': 'Teaching Machines to Code: Neural Markup Generation with Visual Attention', 'model_name': 'I2L-STRIPS', 'paper_url': 'http://arxiv.org/abs/1802.05415v2'}]}, 'subdatasets': [], 'dataset_citations': [], 

In [42]:
datasets[2]["dataset_links"]

[{'url': 'https://paperswithcode.com/sota/optical-character-recognition-on-im2latex-90k',
  'title': 'Papers with Code Leaderboard URL'}]

In [43]:
datasets[2]["dataset"]

'Im2latex-90k'

In [13]:
# 7. synonyms
# all "synonyms" are empty
evaluation_tables[0]['synonyms']

[]

### 1. Does every entity (task, dataset, paper), given as a str, has a specific URL respectively?
- z.B. https://paperswithcode.com/task/image-classification

#### 1.1 Tasks and Subtasks (and subtasks of subtasks etc.)

In [79]:
# 4. task
task = evaluation_tables[0]["task"]
task.lower().replace(" ", "-")


'optical-character-recognition'

In [111]:
tasks_no_url = []
subtasks_no_url = []
prefix = "https://paperswithcode.com/task/"

def url_not_valid(prefix, surfix):
    status = requests.get(prefix+surfix.lower().replace(" ", "-")).status_code
    if status == 404:
        return True

In [112]:
# This only considers Task and their direct Subtasks, but no sub-subtasks -- needs further correction
for element in tqdm(evaluation_tables):
    task_name = element["task"]    
    if url_not_valid(prefix, task_name):
        tasks_no_url.append(task_name)
        
    subtasks_list = element["subtasks"]
    for subtask in subtasks_list:
        subtask_name = subtask["task"]
        if url_not_valid(prefix, subtask_name):
            subtasks_no_url.append({task_name: subtask_name})

100%|███████████████████████████████████████| 1280/1280 [11:00<00:00,  1.94it/s]


In [113]:
len(task_no_url)

88

In [115]:
len(subtasks_no_url)

94

In [102]:
evaluation_tables[0]["subtasks"][0].keys()

dict_keys(['categories', 'subtasks', 'source_link', 'task', 'description', 'datasets', 'synonyms'])

In [107]:
evaluation_tables[0]["subtasks"][0]["subtasks"]

[{'categories': [],
  'subtasks': [],
  'source_link': None,
  'task': 'Active Object Detection',
  'description': 'Active Learning for Object Detection',
  'datasets': [{'dataset_links': [{'url': 'https://paperswithcode.com/sota/active-object-detection-on-coco',
      'title': 'Papers with Code Leaderboard URL'}],
    'description': '',
    'sota': {'metrics': ['AP'],
     'rows': [{'metrics': {'AP': '(7.3, 13.8, 16.9, 19.1, 20.8) on 2% ~ 10%'},
       'paper_date': '2021-04-06',
       'model_links': [],
       'uses_additional_data': False,
       'code_links': [{'url': 'https://github.com/yuantn/MI-AOD',
         'title': 'yuantn/MI-AOD'}],
       'paper_title': 'Multiple instance active learning for object detection',
       'model_name': 'RetinaNet',
       'paper_url': 'https://arxiv.org/abs/2104.02324v1'}]},
    'subdatasets': [],
    'dataset_citations': [],
    'dataset': 'COCO'},
   {'dataset_links': [{'url': 'https://paperswithcode.com/sota/active-object-detection-on-pascal